# **Spelling Recommender for the Herman Melville novel, Moby Dick**

## **Analyzing Moby Dick**

In [ ]:
import nltk
import pandas as pd
import numpy as np
import re

In [ ]:
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/jovyan/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# If you would like to work with the raw text you can use 'moby_raw'
with open('moby.txt', 'r') as f:
    moby_raw = f.read()

moby_tokens = nltk.word_tokenize(moby_raw)
text1 = nltk.Text(moby_tokens)

In [ ]:
type(moby_tokens)

list

In [ ]:
type(text1)

nltk.text.Text

How many tokens (words and punctuation symbols) are in text1?

In [ ]:
def example_one():
    
    return len(nltk.word_tokenize(moby_raw)) # or alternatively len(text1)

example_one()

254989

How many unique tokens (unique words and punctuation) does text1 have?

In [ ]:
def example_two():
    
    return len(set(nltk.word_tokenize(moby_raw))) # or alternatively len(set(text1))

example_two()

20755

After lemmatizing the verbs, how many unique tokens does text1 have?

In [ ]:
from nltk.stem import WordNetLemmatizer

def example_three():

    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w,'v') for w in text1]

    return len(set(lemmatized))

example_three()

16900

What is the lexical diversity of the given text input? (i.e. ratio of unique tokens to the total number of tokens)

In [ ]:
def answer_one():
    
    
    return (len(set(nltk.word_tokenize(moby_raw))))/len(nltk.word_tokenize(moby_raw))

answer_one()

0.08139566804842562

What percentage of tokens is 'whale' or 'Whale'?

In [ ]:
def answer_two():
    
    
    return len([w for w in nltk.word_tokenize(moby_raw) if w=='whale'or w=='Whale']) / len(nltk.word_tokenize(moby_raw))

answer_two()

What are the 20 most frequently occurring (unique) tokens in the text? What is their frequency?

In [ ]:
def answer_three():
    tokens = nltk.word_tokenize(moby_raw)
    dist = nltk.FreqDist(tokens)

    return dist.most_common(20)

answer_three()

What tokens have a length of greater than 5 and frequency of more than 150?

In [ ]:
def answer_four():

    tokens = nltk.word_tokenize(moby_raw)
    dist = nltk.FreqDist(tokens)
    
    return [w for w,v in zip(dist.keys(), dist.values()) if len(w)>5 and v>150]

answer_four()

['through',
 'almost',
 'whales',
 'before',
 'without',
 'should',
 'little',
 'seemed',
 'though',
 'himself',
 'Captain',
 'Queequeg',
 'Pequod',
 'Starbuck']

Find the longest word in text1 and that word's length.

In [ ]:
def answer_five():
    tokens = nltk.word_tokenize(moby_raw)
    tokens_uni = set(tokens)
    longest_word = max(tokens_uni, key=len)
    result = (longest_word, len(longest_word))
    
    return result

answer_five()

("twelve-o'clock-at-night", 23)

What unique words have a frequency of more than 2000? What is their frequency?

In [ ]:
def answer_six():
    
    tokens = nltk.word_tokenize(moby_raw)
    dist = nltk.FreqDist(tokens)
    return [(v,w) for w,v in zip(dist.keys(), dist.values()) if v>2000 and w.isalpha()]

answer_six()

[(4545, 'a'),
 (4515, 'to'),
 (3908, 'in'),
 (6010, 'and'),
 (2097, 'I'),
 (2459, 'his'),
 (13715, 'the'),
 (6513, 'of'),
 (2196, 'it'),
 (2978, 'that')]

What is the average number of tokens per sentence?

In [ ]:
def answer_seven():
    c=0
    sent_tokens = nltk.sent_tokenize(moby_raw)
    for s in sent_tokens:
        c=c+ len(nltk.word_tokenize(s))
        
    
    return c/len(sent_tokens)

answer_seven()

25.881952902963864

What are the 5 most frequent parts of speech in this text? What is their frequency?

In [ ]:
def answer_eight():
    
    tokens = nltk.word_tokenize(moby_raw)
    tags = nltk.pos_tag(tokens)
    dist = nltk.FreqDist([tag for (word, tag) in tags])
    return dist.most_common(5)

answer_eight()

[('NN', 32730), ('IN', 28657), ('DT', 25867), (',', 19204), ('JJ', 17620)]

## **Framing the Spelling Recommender for Moby Dick**

The recommenders deliver recommendations for the three default words provided: `['cormulent', 'incendenece', 'validrate']`.

In [ ]:
from nltk.corpus import words
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams

In [ ]:
from nltk.corpus import words
correct_spellings = words.words()

This Recommender delivers the recommendations for the three default words provided above using the following distance metric:

**[Jaccard distance](https://en.wikipedia.org/wiki/Jaccard_index) on the trigrams of the two words.**

In [ ]:
entries=['cormulent', 'incendenece', 'validrate']

In [ ]:
def answer_nine(entries, gram_number):
    
    arr=[]
    for entry in entries:
        spellings = [s for s in correct_spellings if s.startswith(entry[0])]
        df= pd.DataFrame(columns=['word', 'Dist_'])
        dist_df1=[]
        dist_df2=[]
        for word in spellings:
            dist_df1.append(word)
            dist_df2.append(jaccard_distance(set(ngrams(entry, gram_number)), set(ngrams(word, gram_number))))
        x= list(zip(dist_df2,dist_df1))
        y= min(x)
        arr.append(y[1])
    return arr
answer_nine(entries,3)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: generator 'ngrams' raised StopIteration
  # This is added back by InteractiveShellApp.init_path()


['corpulent', 'indecence', 'validate']

This recommender delivers the recommendations for the three default words provided above using the following distance metric:

**[Jaccard distance](https://en.wikipedia.org/wiki/Jaccard_index) on the 4-grams of the two words.**

In [ ]:
def answer_ten():

    return answer_nine(entries,4)
    
answer_ten()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: generator 'ngrams' raised StopIteration
  # This is added back by InteractiveShellApp.init_path()


['cormus', 'incendiary', 'valid']

This recommender delivers the recommendations for the three default words provided above using the following distance metric:

**[Edit distance on the two words with transpositions.](https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance)**

In [ ]:
def answer_eleven(entries):

    arr=[]
    for entry in entries:
        spellings = [s for s in correct_spellings if s.startswith(entry[0])]
        df= pd.DataFrame(columns=['word', 'Dist_'])
        dist_df1=[]
        dist_df2=[]
        for word in spellings:
            dist_df1.append(word)
            dist_df2.append(edit_distance(entry, word))
        x= list(zip(dist_df2,dist_df1))
        y= min(x)
        arr.append(y[1])
    return arr
    
answer_eleven(entries)

['corpulent', 'intendence', 'validate']